In [276]:
#import git
#git.Git("../").clone("https://github.com/sonos/nlu-benchmark")


In [277]:
import os
import json
import numpy as np
from tensorflow import keras

In [278]:
DATA_PATH = './nlu-benchmark/2017-06-custom-intent-engines/'
DEFAULT_SLOT_LABEL = 'unspecified'

In [279]:
cls_labels = [ domain for domain in os.listdir(DATA_PATH) if os.path.isdir(DATA_PATH  + domain) ]

In [280]:
cls_train_x = []
cls_train_y = []
cls_val_x = []
cls_val_y = []

Weather_seq_l_train_x = []
Weather_seq_l_train_y = []
Weather_seq_l_val_x = []
Weather_seq_l_val_y = []
weather_seq_label_set = set()

In [281]:
def load_data( domain, dataset_type, full):
    file_path = os.path.join(DATA_PATH, "{}/{}_{}{}.json".format(domain, dataset_type, domain, full))
    with open(file_path, 'r') as filename:
        print(filename)
        dataset = json.load(filename)[domain]
    
    cls_x = []
    cls_y = []
    seq_l_x = []
    seq_l_y = []
    for element in dataset:
        tokenized_sentence = []
        seq_labels = []
        for piece in element['data']:
            words = piece['text'].strip().split()
            tokenized_sentence += words
            for token in words:
                if 'entity' in piece.keys():
                    seq_labels += [piece['entity']]
                else:
                    seq_labels.append(DEFAULT_SLOT_LABEL)
        cls_x.append(tokenized_sentence)
        cls_y.append(domain)
        seq_l_x.append(tokenized_sentence)
        seq_l_y.append(seq_labels)
    return cls_x, cls_y, seq_l_x, seq_l_y
        
        
            
        
    

In [282]:
for domain in cls_labels:
    cls_x, cls_y, seq_l_train_x, seq_l_train_y = load_data(domain, 'train', '_full')
    cls_train_x += cls_x 
    cls_train_y += cls_y
    
    cls_x, cls_y, seq_l_val_x, seq_l_val_y = load_data(domain, 'validate', '')
    cls_val_x += cls_x
    cls_val_y += (cls_y)
    
    if domain == 'GetWeather':
        Weather_seq_l_train_x = seq_l_train_x
        Weather_seq_l_train_y = seq_l_train_y
        Weather_seq_l_val_x = seq_l_val_x
        Weather_seq_l_val_y = seq_l_val_y
        for seq_label in Weather_seq_l_train_y:
            weather_seq_label_set |= set(seq_label)
        for seq_label in Weather_seq_l_val_y:
            weather_seq_label_set |= set(seq_label)            
    
weather_seq_label_list = list(weather_seq_label_set)
weather_seq_label_list.remove('unspecified')
weather_seq_label_list.insert(0, 'unspecified')   

<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/RateBook/train_RateBook_full.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/RateBook/validate_RateBook.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/SearchCreativeWork/train_SearchCreativeWork_full.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/SearchCreativeWork/validate_SearchCreativeWork.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/GetWeather/train_GetWeather_full.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/GetWeather/validate_GetWeather.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='./nlu-benchmark/2017-06-custom-intent-engines/BookRestaurant/train_BookRestaurant_full.json' mode='r' encoding='UTF-8'>
<_io.TextIOWrap

In [283]:
def vectorization(train_x, train_y, valid_x, valid_y, labels, task = 'classification', max_len=64):

    special_tokens = ['<pad>', '<unk>']
    word2ind = {}
    ind2word = {}
    
    def add_word(word2ind, ind2word, word):
        if word in word2ind.keys():
            return
        word2ind[word] = len(ind2word)
        ind2word[len(ind2word)] = word
    for token in special_tokens:
        add_word(word2ind, ind2word, token)
        
    for sent in train_x:
        for word in sent:
            add_word(word2ind, ind2word, word)

    train_x_ids = []
    for sent in train_x:
        sent_ids = []
        for word in sent:
            sent_ids.append(word2ind.get(word, word2ind['<unk>']))
        train_x_ids.append(sent_ids)
        
    train_x_ids = keras.preprocessing.sequence.pad_sequences(train_x_ids, maxlen = max_len, padding = 'post', value = word2ind['<pad>'])
     
    val_x_ids = []
    for sent in valid_x:
        sent_ids = []
        for word in sent:
            sent_ids.append(word2ind.get(word, word2ind['<unk>']))
        val_x_ids.append(sent_ids)
        
    val_x_ids = keras.preprocessing.sequence.pad_sequences(val_x_ids, maxlen = max_len, padding = 'post', value = word2ind['<pad>'])
       
        
        


    if task == 'classification':
        train_y_ids = [labels.index(label) for label in train_y]
        val_y_ids = [labels.index(label) for label in valid_y]
    #classification data vectorization

    elif task == 'slot_labelling':
        train_y_ids =  [[labels.index(label) for label in sent] for sent in train_y]
        train_y_ids = keras.preprocessing.sequence.pad_sequences(train_y_ids, maxlen = max_len, padding = 'post', value = word2ind['<pad>'])
        val_y_ids =  [[labels.index(label) for label in sent] for sent in valid_y]
        val_y_ids = keras.preprocessing.sequence.pad_sequences(val_y_ids, maxlen = max_len, padding = 'post', value = word2ind['<pad>'])
          

    return word2ind, train_x_ids, train_y_ids, val_x_ids,  val_y_ids

In [284]:
cls_word2ind, cls_train_x_ids, cls_train_y_ids, cls_val_x_ids, cls_val_y_ids = vectorization(cls_train_x, cls_train_y,cls_val_x, \
                                                                                             cls_val_y, cls_labels, 'classification' )
Weather_seq_word2ind, Weather_seq_l_train_x_ids, Weather_seq_l_train_y_ids, \
    Weather_seq_l_val_x_ids, Weather_seq_l_val_y_ids =  vectorization(Weather_seq_l_train_x, Weather_seq_l_train_y,\
                                                                      Weather_seq_l_val_x, Weather_seq_l_val_y, weather_seq_label_list, 'slot_labelling')

Weather_seq_sample_weights = np.ones(Weather_seq_l_train_y_ids.shape)
for i, sequence in enumerate(Weather_seq_l_train_y_ids):
    for j, label in enumerate(sequence):
        if label == weather_seq_label_list.index('unspecified'):
            Weather_seq_sample_weights[i][j] = 0.1


In [285]:
import pickle

cls_data = {'word2ind': cls_word2ind,
           'label_list': cls_labels,
           'train_x': cls_train_x_ids,
           'train_y': cls_train_y_ids,
           'val_x': cls_val_x_ids,
           'val_y': cls_val_y_ids}

weather_seq_data = {'word2ind': Weather_seq_word2ind,
           'label_list': weather_seq_label_list,
           'sample_weight': Weather_seq_sample_weights,
           'train_x': Weather_seq_l_train_x_ids,
           'train_y': Weather_seq_l_train_y_ids,
           'val_x': Weather_seq_l_val_x_ids,
           'val_y': Weather_seq_l_val_y_ids}
pickle.dump(cls_data, open('cls_data.pickle', 'wb'))
pickle.dump(weather_seq_data, open('weather_seq_data.pickle', 'wb'))

In [286]:
weather_seq_label_list

['unspecified',
 'country',
 'condition_temperature',
 'city',
 'state',
 'spatial_relation',
 'timeRange',
 'geographic_poi',
 'current_location',
 'condition_description']

In [287]:
Weather_seq_l_train_y_ids[0]
Weather_seq_l_train_y[0]

['unspecified',
 'unspecified',
 'unspecified',
 'unspecified',
 'unspecified',
 'timeRange',
 'timeRange',
 'unspecified',
 'geographic_poi',
 'geographic_poi',
 'geographic_poi',
 'geographic_poi',
 'geographic_poi',
 'geographic_poi',
 'geographic_poi',
 'unspecified']

In [288]:
len(Weather_seq_l_train_y)

2000

In [289]:
Weather_seq_l_train_x_ids


array([[  2,   3,   4, ...,   0,   0,   0],
       [ 18,  19,   6, ...,   0,   0,   0],
       [ 33,  19,  34, ...,   0,   0,   0],
       ...,
       [ 18,  19, 153, ...,   0,   0,   0],
       [162, 100,   4, ...,   0,   0,   0],
       [  2,  45,   4, ...,   0,   0,   0]], dtype=int32)

In [290]:
Weather_seq_sample_weights[0]


array([0.1, 0.1, 0.1, 0.1, 0.1, 1. , 1. , 0.1, 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [291]:
Weather_seq_l_val_y_ids[2]

array([0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [292]:
Weather_seq_l_val_y[2]

['unspecified',
 'unspecified',
 'unspecified',
 'unspecified',
 'unspecified',
 'unspecified',
 'city']

In [293]:
Weather_seq_l_val_x[2]

['Tell', 'me', 'the', 'weather', 'forecast', 'for', 'Gibsland']

In [294]:
Weather_seq_l_train_x_ids

array([[  2,   3,   4, ...,   0,   0,   0],
       [ 18,  19,   6, ...,   0,   0,   0],
       [ 33,  19,  34, ...,   0,   0,   0],
       ...,
       [ 18,  19, 153, ...,   0,   0,   0],
       [162, 100,   4, ...,   0,   0,   0],
       [  2,  45,   4, ...,   0,   0,   0]], dtype=int32)

In [295]:
Weather_seq_l_train_y_ids[0:5]

array([[0, 0, 0, 0, 0, 6, 6, 0, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 9, 6, 6, 6, 6, 6, 6, 6, 0, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 9, 5, 0, 8, 8, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 6, 6, 6, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [296]:
Weather_seq_l_val_y_ids.shape

(100, 64)

In [297]:
Weather_seq_l_train_x[0]

['What',
 'will',
 'the',
 'weather',
 'be',
 'this',
 'year',
 'in',
 'Horseshoe',
 'Lake',
 'State',
 'Fish',
 'and',
 'Wildlife',
 'Area',
 '?']

In [298]:
Weather_seq_word2ind

{'<pad>': 0,
 '<unk>': 1,
 'What': 2,
 'will': 3,
 'the': 4,
 'weather': 5,
 'be': 6,
 'this': 7,
 'year': 8,
 'in': 9,
 'Horseshoe': 10,
 'Lake': 11,
 'State': 12,
 'Fish': 13,
 'and': 14,
 'Wildlife': 15,
 'Area': 16,
 '?': 17,
 'Will': 18,
 'it': 19,
 'sunny': 20,
 'one': 21,
 'hundred': 22,
 'thirty': 23,
 'five': 24,
 'days': 25,
 'from': 26,
 'now': 27,
 'Monterey': 28,
 'Bay': 29,
 'National': 30,
 'Marine': 31,
 'Sanctuary': 32,
 'Is': 33,
 'supposed': 34,
 'to': 35,
 'rain': 36,
 'nearby': 37,
 'my': 38,
 'current': 39,
 'location': 40,
 'at': 41,
 '0': 42,
 "o'clock": 43,
 'what': 44,
 'is': 45,
 'forecast': 46,
 'starting': 47,
 'on': 48,
 'september': 49,
 '1,': 50,
 '2039': 51,
 'for': 52,
 'chillier': 53,
 'conditions': 54,
 'AK': 55,
 'how': 56,
 'cold': 57,
 'Princeton': 58,
 'Junction': 59,
 'Nationalpark': 60,
 'Nevado': 61,
 'Tres': 62,
 'Cruces': 63,
 'mar.': 64,
 '4th,': 65,
 '2020': 66,
 'wind': 67,
 'speed': 68,
 'Tiplersville': 69,
 ',': 70,
 'South': 71,
 'Suda